In [1]:
import requests
import pandas as pd
import arrow
import datetime
import warnings

In [2]:
#company =[ "CIPLA","TCS","INFY","INFRATEL","ITC","HCLTECH","VEDL","WIPRO","BRITANNIA","UPL","RELIANCE","TECHM","M_M","SUNPHARMA","NESTLEIND",
#"HINDUNILVR","DRREDDY","TATASTEEL","BHARTIARTL","NTPC","ONGC","TITAN","POWERGRID","HINDALCO","ASIANPAINT","TATAMOTORS","SHREECEM",
#"GRASIM","IOC","HEROMOTOCO","BAJAJFINSV","ADANIPORTS","EICHERMOT","COALINDIA","JSWSTEEL","KOTAKBANK","LT","GAIL","HDFCBANK",
#"SBIN","BAJAJ-AUTO","BAJFINANCE","BPCL","HDFC","MARUTI","ULTRACEMCO","AXISBANK","ICICIBANK","ZEEL","INDUSINDBK" ]

company = ["ICICIBANK","ZEEL","INDUSINDBK"]

In [3]:
spanb = 10 #Span for Bollinger Bands
spanv = 5  #Span for VWAP
span = 14  #Span for ATR and RSI
stds = 2   #No. of standard deviations for Bollinger Bands

In [4]:
def VWAP(ndata, spanv):
    num = (((ndata['High']+ndata['Low']+ndata['Close'])/3)*ndata['Volume']).rolling(spanv).sum()
    den = ndata['Volume'].rolling(spanv).sum()
    return num/den

In [5]:
def BB(ndata, spanb, stds): 
    ndata['UBB'] = ndata.Close.rolling(spanb).mean() + stds*ndata.Close.rolling(spanb).std()
    ndata['MBB'] = ndata.Close.rolling(spanb).mean()
    ndata['LBB'] = ndata.Close.rolling(spanb).mean() - stds*ndata.Close.rolling(spanb).std()
    return ndata

In [6]:
def ATR(ndata, span):
    return pd.DataFrame([ndata['High']-ndata['Low'],abs(ndata['High']-ndata['Close'].rolling(2).sum()+ndata['Close']),abs(ndata['Low']-ndata['Close'].rolling(2).sum()+ndata['Close'])]).max(axis = 0).rolling(span).mean()

In [7]:
def RSI(got, span):
    got['Gain'] = got['Close']-(got['Close'].rolling(2).sum()-got['Close'])
    got['Gain'][got['Gain']<0]=0
    got['Loss'] = -(got['Close']-(got['Close'].rolling(2).sum()-got['Close']))
    got['Loss'][got['Loss']<0]=0
    got['Average Gain'] = float("NaN")
    got['Average Loss'] = float("NaN")
    got['Average Gain'][14] = got['Gain'][1:15].mean()
    got['Average Loss'][14] = got['Loss'][1:15].mean()
    
    for j in range(15,len(got)):
        got['Average Gain'][j] = (got['Average Gain'][j-1]*13 + got['Gain'][j])/14
        got['Average Loss'][j] = (got['Average Loss'][j-1]*13 + got['Loss'][j])/14
        
    got['RSI'] = 100 - 100 / (1+ got['Average Gain'] / got['Average Loss'])
    return got['RSI'][0:len(got)]

In [8]:
for i in company :
    print(i)
    data = pd.read_csv('C:/Users/user/Stockomation/Github/Data/Updated/NIFTY 50/1MIN/'+i+'.csv')
    data.rename(columns={'open': 'Open', 'high' : 'High', 'low' : 'Low', 'close': 'Close', 'volume' : 'Volume'}, inplace = True)
    warnings.filterwarnings("ignore")
    data['RSI'] = RSI(data, span)
    data.set_index('Datetime', inplace = True)
    data['VWAP'] = VWAP(data, spanv)
    data = BB(data, spanb, stds)
    data['ATR'] = ATR(data, span)
    data.to_csv('C:/Users/user/Stockomation/Github/Data/Updated/NIFTY 50/1MIN/'+i+'.csv')

ICICIBANK
ZEEL
INDUSINDBK
